In [1]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

NVIDIA A100-PCIE-40GB, 40960 MiB, 28522 MiB


In [2]:
import torch
import os
from diffusers import StableDiffusionPipeline
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer

SEED = None
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
generator = None if SEED is None else torch.Generator(
            device=DEVICE).manual_seed(SEED)

pretrained_model_name_or_path = "runwayml/stable-diffusion-v1-5"
# choose pivotal_reconstruction or factor_optimization
learned_embeds_path = "/root/autodl-tmp/textual_inversion/trained_embeddings/custom_cat/pivotal_reconstruction/learned_embeds.bin"
all_embedding_path = os.path.dirname(learned_embeds_path)
dataset_path, _ = os.path.split(all_embedding_path)
_, dataset_name = os.path.split(dataset_path)

tokenizer = CLIPTokenizer.from_pretrained(
    pretrained_model_name_or_path,
    subfolder="tokenizer",
)
text_encoder = CLIPTextModel.from_pretrained(
    pretrained_model_name_or_path, subfolder="text_encoder", torch_dtype=torch.float16
)

/root/miniconda3/envs/dushian/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
loaded_learned_embeds = torch.load(learned_embeds_path, map_location="cpu")
  
# separate token and the embeds
trained_token_content = list(loaded_learned_embeds.keys())[0]
trained_token_style = list(loaded_learned_embeds.keys())[1]
embeds_content = loaded_learned_embeds[trained_token_content]
embeds_style = loaded_learned_embeds[trained_token_style]
print("placeholder token for dataset {}: content: {}, style: {}".format(dataset_name, trained_token_content, 
                                                                        trained_token_style))

# cast to dtype of text_encoder
dtype = text_encoder.get_input_embeddings().weight.dtype
embeds_content.to(dtype)
embeds_style.to(dtype)

# add the token in tokenizer
num_added_tokens_content = tokenizer.add_tokens(trained_token_content)
if num_added_tokens_content == 0:
    raise ValueError(f"The tokenizer already contains the token {trained_token_content}. "
                     "Please pass a different `content token` that is not already in the tokenizer.")
num_added_tokens_style = tokenizer.add_tokens(trained_token_style)
if num_added_tokens_style == 0:
    raise ValueError(f"The tokenizer already contains the token {trained_token_style}. "
                     "Please pass a different `style token` that is not already in the tokenizer.")

# resize the token embeddings
text_encoder.resize_token_embeddings(len(tokenizer))

# get the id for the token and assign the embeds
token_id_content = tokenizer.convert_tokens_to_ids(trained_token_content)
token_id_style = tokenizer.convert_tokens_to_ids(trained_token_style)
text_encoder.get_input_embeddings().weight.data[token_id_content] = embeds_content
text_encoder.get_input_embeddings().weight.data[token_id_style] = embeds_style

pipe = StableDiffusionPipeline.from_pretrained(pretrained_model_name_or_path,
                                               torch_dtype=torch.float16, 
                                               text_encoder=text_encoder, 
                                               tokenizer=tokenizer).to(DEVICE)

placeholder token for dataset custom_cat: content: <custom_cat_content>, style: <custom_cat_style>


/root/miniconda3/envs/dushian/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [8]:
# USE both token
prompt = "a photo of {} in the classroom in the style of {}".format(trained_token_content, trained_token_style)
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, generator=generator).images[0]

image_dir = os.path.join(all_embedding_path, "images")
os.makedirs(image_dir, exist_ok=True)
image_path = os.path.join(image_dir, "{}.png".format(prompt))
image.save(image_path)

100%|██████████| 50/50 [00:05<00:00,  9.90it/s]


In [27]:
# USE only content token
prompt = "a photo of Elmo holding {}".format(trained_token_content)
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, generator=generator).images[0]

image_dir = os.path.join(all_embedding_path, "images")
os.makedirs(image_dir, exist_ok=True)
image_path = os.path.join(image_dir, "{}.png".format(prompt))
image.save(image_path)

100%|██████████| 50/50 [00:05<00:00,  9.97it/s]


In [26]:
# USE only style token
prompt = "a painting of a cat in the style of {}".format(trained_token_style)
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, generator=generator).images[0]

image_dir = os.path.join(all_embedding_path, "images")
os.makedirs(image_dir, exist_ok=True)
image_path = os.path.join(image_dir, "{}.png".format(prompt))
image.save(image_path)

100%|██████████| 50/50 [00:05<00:00,  9.81it/s]
